# 5. MDP를 모를 때 밸류 평가하기
바닥부터 배우는 강화 학습 5장에 있는 코드를 참고 했습니다.

In [19]:
import random

from tqdm import trange

# 5.1 몬테카를로 학습

- GridWorld 클래스

In [14]:
class GridWorld:
    def __init__(self):
        self.x = 0
        self.y = 0
    
    def step(self, a):
        if a == 0:
            self.move_right()
        elif a == 1:
            self.move_left()
        elif a == 2:
            self.move_up()
        elif a == 3:
            self.move_down()
        
        reward = -1
        done = self.is_done()
        return (self.x, self.y), reward, done
    
    def move_right(self):
        self.y = min(self.y + 1, 3)
    
    def move_left(self):
        self.y = max(self.y - 1, 0)
    
    def move_up(self):
        self.x = max(self.x - 1, 0)
    
    def move_down(self):
        self.x = min(self.x + 1, 3)
    
    def is_done(self):
        return (self.x == 3 and self.y == 3)
    
    def get_state(self):
        return (self.x, self.y)
    
    def reset(self):
        self.x = 0
        self.y = 0
        return (self.x, self.y)

- Agent 클래스

In [3]:
class Agent:
    def __init__(self):
        pass
    
    def select_action(self):
        coin = random.random()
        if coin < 0.25:
            action = 0
        elif coin < 0.5:
            action = 1
        elif coin < 0.75:
            action = 2
        else:
            action = 3
        return action

- 초기 환경

In [16]:
env = GridWorld()
agent = Agent()
data = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
gamma = 1.0
alpha = 0.0001

- 5만번 에피소드 진행

In [24]:
for k in trange(50000):
    done = False
    history = []
    # 에피소드 1회 진행
    while not done:
        action = agent.select_action()
        (x, y), reward, done = env.step(action)
        history.append((x, y, reward))
    # env 초기화
    env.reset()

    # 에피소드 종료 후 테이블 업데이트
    cum_reward = 0
    for transition in history[::-1]:  # history의 뒤쪽부터 차례차례 리턴을 계산
        x, y, reward = transition
        data[x][y] = data[x][y] + alpha * (cum_reward - data[x][y])
        cum_reward = cum_reward + gamma * reward

100%|██████████| 50000/50000 [00:06<00:00, 7604.82it/s]


- 학습이 끝난 후 데이터 출력해보기

In [25]:
for row in data:
    print(row)

[-61.316625703755335, -59.46039270775394, -57.67621919458881, -54.19033058917396]
[-59.41062545856224, -57.39176135297013, -52.482414647454505, -47.62986165427043]
[-55.80015061080171, -51.32209061773418, -42.04046172058729, -31.72585696942092]
[-52.129614422146965, -46.01930125916076, -31.046392777092905, 0.0]


# 5.2 Temporal Difference 학습

- 초기 환경

In [28]:
env = GridWorld()
agent = Agent()
data = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]
gamma = 1.0
alpha = 0.0001

- 5만번 에피소드 진행

In [32]:
for k in trange(50000):
    done = False
    # 에피소드 1회 진행
    while not done:
        x, y = env.get_state()
        action = agent.select_action()
        (x_prime, y_prime), reward, done = env.step(action)
        x_prime, y_prime = env.get_state()

        # 한 번의 step이 진행되자 마자 바로 테이블 업데이트
        data[x][y] = data[x][y] + alpha * (reward + gamma * data[x_prime][y_prime] - data[x][y])
    env.reset()

100%|██████████| 50000/50000 [00:07<00:00, 6837.77it/s]


- 학습이 끝난 후 데이터 출력해보기

In [33]:
for row in data:
    print(row)

[-33.237589061953194, -31.755766060544918, -29.628795972055563, -27.974517522812]
[-31.833274966098998, -29.971190869296443, -26.965289617109057, -24.29577375455628]
[-29.691807263455573, -27.025037581773542, -21.977859135836937, -16.067124156734142]
[-28.00424244328802, -24.3318562989472, -16.00934649547561, 0]
